# Tema 10: Procesamiento del Lenguaje Natural (IV) - WordNet
En este cuaderno vamos a aprender qué es WordNet y algunas de las cosas más útiles que se pueden hacer con esta librería, como explorar sinónimos, obtener traducciones contextuales, explorar relaciones de hiperonimia y holonimia, y abordar algunas de las cuestiones más ambiciosas del PLN: la expansión de búsquedas, el cálculo de la distancia semántica o la desambiguación.

En este cuaderno no hay ejercicios al final, pero hay mucho espacio a la experimentación con otras palabras, otros synsets, otros idiomas... a lo largo de toda la lección.

In [1]:
from nltk.corpus import wordnet # Importamos WordNet
from IPython.display import HTML, display, Image # Importamos una serie de clases y funciones útiles

def display_table(data, headers=None, caption=None):
    ''' Una función para imprimir tablas en formato HTML, solo por estética
    '''
    html = ["<table align=\"left\">"]
    
    if caption:
        html += ["<caption>{}</caption>".format(caption)]
        
    if headers:
        html += ["<tr>"] + ["<th>{}</th>".format(h) for h in headers] + ["</tr>"]
    
    for row in data:
        html += ["<tr>"]
        html += ["<td>{}</td>".format(it) for it in row]
        html += ["</tr>"]

    html.append("</table>")
    display(HTML(''.join(html)))    

## WordNet

[WordNet](https://wordnet.princeton.edu/) es una red de conceptos que contiene información codificada manualmente sobre sustantivos, verbos, adjetivos y adverbios en inglés; los términos que representan un mismo concepto están agrupados en _synsets_ y son estos elementos los que constituyen los nodos de la red.

WordNet se creó en el Laboratorio de Ciencia Cognitiva de la Universidad de Princeton en 1985 bajo la dirección del profesor de psicología George Armitage Miller (1920-2012).

Para que nos hagamos una idea de la cantidad de información que contiene ([más números](http://wordnet.princeton.edu/wordnet/man/wnstats.7WN.html)):

In [2]:
n_nouns = len(list(wordnet.all_synsets(pos=wordnet.NOUN)))
n_verbs = len(list(wordnet.all_synsets(pos=wordnet.VERB)))
n_adj = len(list(wordnet.all_synsets(pos=wordnet.ADJ)))
n_adv = len(list(wordnet.all_synsets(pos=wordnet.ADV)))

print("¿Qué hay dentro de WordNet?")
print("Hay {} nombres.".format(n_nouns))
print("Hay {} verbos.".format(n_verbs))
print("Hay {} adjetivos.".format(n_adj))
print("Hay {} adverbios.".format(n_adv))

¿Qué hay dentro de WordNet?
Hay 82115 nombres.
Hay 13767 verbos.
Hay 18156 adjetivos.
Hay 3621 adverbios.


(Observa lo que hace el método `format()`: recibe como parámetro una variable, y se aplica a una string que debe contener `{}`. En vez de estos corchetes, imprime la variable, y como ves no hace falta convertirla a string previamente.)
## Synsets
Un _synset_ (_syonyms set_) es un conjunto de sinónimos, palabras de la misma categoría gramatical que hacen referencia a la misma realidad extralingüística y por lo tanto pueden ser intercambiadas en un texto sin afectar al significado. Son elementos semánticamente equivalentes. Así, ocurrirá que las palabras polisémicas aparecerán en muchos synsets diferentes.

Podemos hacer una búsqueda de uno de estos synsets pasándole el término de búsqueda que elijamos a la función `synsets()` como string:

In [3]:
my_synsets = wordnet.synsets('bed')
print("Esta palabra pertenece a", len(my_synsets), "synsets distintos:")

print(my_synsets)
print(type(my_synsets[0]))

Esta palabra pertenece a 13 synsets distintos:
[Synset('bed.n.01'), Synset('bed.n.02'), Synset('bed.n.03'), Synset('bed.n.04'), Synset('seam.n.03'), Synset('layer.n.01'), Synset('bed.n.07'), Synset('bed.n.08'), Synset('bed.v.01'), Synset('bed.v.02'), Synset('bed.v.03'), Synset('sleep_together.v.01'), Synset('go_to_bed.v.01')]
<class 'nltk.corpus.reader.wordnet.Synset'>


Nos devuelve una lista de objetos de tipo Synset, como podemos ver. Estos objetos tienen varios atributos interesantes para explorar, como `name` o `definition`:

In [4]:
data = []
for synset in my_synsets:
    data.append([synset.name(), synset.definition()])

display_table(data, ["Synset", "Definición"])

Synset,Definición
bed.n.01,a piece of furniture that provides a place to sleep
bed.n.02,a plot of ground in which plants are growing
bed.n.03,a depression forming the ground under a body of water
bed.n.04,(geology) a stratum of rock (especially sedimentary rock)
seam.n.03,a stratum of ore or coal thick enough to be mined with profit
layer.n.01,single thickness of usually some homogeneous substance
bed.n.07,the flat surface of a printing press on which the type form is laid in the last stage of producing a newspaper or magazine or book etc.
bed.n.08,a foundation of earth or rock supporting a road or railroad track
bed.v.01,furnish with a bed
bed.v.02,place (plants) in a prepared bed of soil


Podemos quedarnos con uno de ellos y explorar la cantidad de información que ofrece WordNet una vez que hemos encontrado el synset que nos interesa (pero puedes probar con otros: `my_synsets[0]`, `my_synsets[1]`...):

In [5]:
my_synset = my_synsets[11]
print("synset.name: {}".format(my_synset.name()))
print("synset.definition: {}".format(my_synset.definition()))

print("synset.examples:")
for example in my_synset.examples():
    print("\t + {}".format(example))

print("synset.lemmas:")
for lemma in my_synset.lemmas():
    print("\t + {}".format(lemma.name()))

synset.name: sleep_together.v.01
synset.definition: have sexual intercourse with
synset.examples:
	 + This student sleeps with everyone in her dorm
	 + Adam knew Eve
	 + Were you ever intimate with this man?
synset.lemmas:
	 + sleep_together
	 + roll_in_the_hay
	 + love
	 + make_out
	 + make_love
	 + sleep_with
	 + get_laid
	 + have_sex
	 + know
	 + do_it
	 + be_intimate
	 + have_intercourse
	 + have_it_away
	 + have_it_off
	 + screw
	 + fuck
	 + jazz
	 + eff
	 + hump
	 + lie_with
	 + bed
	 + have_a_go_at_it
	 + bang
	 + get_it_on
	 + bonk


También podemos buscar los lemmas correspondientes a un synset en otros idiomas. Vamos a mostrar aquí solo un ejemplo porque trataremos este tema más adelante. Véamos cuáles son los relacionados con el synset que hemos guardado en la variable `my_synset` de la celda anterior:

In [6]:
languages = sorted(wordnet.langs())
print("Idiomas disponibles: {}".format(', '.join(languages)))

selected_languages = ['eng', 'spa', 'fra'] # Puedes probar con otros idiomas de la lista

data = []
for lang in selected_languages:
    data.append([lang, '</br>'.join(my_synset.lemma_names(lang))])

display_table(data, headers=["lang", "lemmas"])

Idiomas disponibles: als, arb, bul, cat, cmn, dan, ell, eng, eus, fas, fin, fra, glg, heb, hrv, ind, ita, jpn, nld, nno, nob, pol, por, qcn, slv, spa, swe, tha, zsm


lang,lemmas
eng,sleep_togetherroll_in_the_haylovemake_outmake_lovesleep_withget_laidhave_sexknowdo_itbe_intimatehave_intercoursehave_it_awayhave_it_offscrewfuckjazzeffhumplie_withbedhave_a_go_at_itbangget_it_onbonk
spa,joder
fra,aimeravoiravoir_des_relations_sexuellesbaiserbienconnaîtreenconnerenculerfaire_l'amourfourrerfoutreniquersavoir


Claramente, hay que ampliar el vocabulario que contiene en español...
### Categoría gramatical de un synset
En el apartado anterior hemos recuperado todos los synsets a partir de una palabra y nos han aparecido significados correspondientes a sustantivos, verbos, adjetivos... pero se puede afinar un poco más la búsqueda utilizando la PoS:

 - `wordnet.VERB`
 - `wordnet.NOUN`
 - `wordnet.ADJ`
 - `wordnet.ADV`
 
¡Vamos a verlo en acción!

In [7]:
word = "bien" # Puedes probar con otras palabras que correspondan a varias categorías gramaticales
lang = "spa"

synsets_as_noun = wordnet.synsets(word, lang=lang, pos=wordnet.NOUN)
synsets_as_verb = wordnet.synsets(word, lang=lang, pos=wordnet.VERB)
synsets_as_adj = wordnet.synsets(word, lang=lang, pos=wordnet.ADJ)
synsets_as_adv = wordnet.synsets(word, lang=lang, pos=wordnet.ADV)

# Vamos a imprimir los resultados
print("synsets_as_noun: {}".format(synsets_as_noun))

def synset_table(synsets, title):
    data = []
    for synset in synsets:
        data.append([synset.name(), synset.lemma_names(), synset.definition()])
        
    display_table(data, ["Synset", "Lemmas", "Definición"], caption=title)

synset_table(synsets_as_noun, title="Resultados con: wordnet.NOUN")
synset_table(synsets_as_verb, title="Resultados con: wordnet.VERB")
synset_table(synsets_as_adj,  title="Resultados con: wordnet.ADJ")
synset_table(synsets_as_adv, title="Resultados con: wordnet.ADV")

synsets_as_noun: [Synset('commodity.n.01'), Synset('good.n.03'), Synset('sake.n.01'), Synset('good.n.01'), Synset('personal_property.n.01')]


Synset,Lemmas,Definición
commodity.n.01,"['commodity', 'trade_good', 'good']",articles of commerce
good.n.03,"['good', 'goodness']",that which is pleasing or valuable or useful
sake.n.01,"['sake', 'interest']",a reason for wanting something done
good.n.01,['good'],benefit
personal_property.n.01,"['personal_property', 'personal_estate', 'personalty', 'private_property']",movable property (as distinguished from real estate)


Synset,Lemmas,Definición


Synset,Lemmas,Definición
good.s.17,"['good', 'sound']",in excellent physical condition
all_right.s.01,"['all_right', 'fine', 'o.k.', 'ok', 'okay', 'hunky-dory']",being satisfactory or in satisfactory condition


Synset,Lemmas,Definición
well.r.13,['well'],without unusual distress or resentment; with good humor
well.r.06,['well'],favorably; with approval
well.r.12,"['well', 'comfortably']",in financial comfort


Como era de esperar, la tabla de verbos ha salido vacía porque en ningún caso "bien" actúa como verbo.
### Lo interesante de los synsets
Lo interesante de los synsets es que permiten referirse a un significado sin ambigüedades. A los ordenadores se les da muy mal resolver ambigüedades, generalmente se les da muy mal todo lo que humanos hacemos con relativa facilidad (entender un mensaje, reconocer imágenes...), pero realizan muy eficazmente tareas que a nosotros nos cuestan mucho tiempo (búsquedas, ordenación...).

Los ordenadores querrían ver los textos de esta forma, **sin ambigüedades**:

```
El perro ladra ---> El dog.n.01 bark.v.04
```
así podrían entenderlo y podríamos hacer inferencias sin equivocarnos.

## Lemmas

No debe confundirse un **synset** con un **lemma**, tal y como los identifica WordNet. Recordemos que:
 
 * un **synset** está asociado a un significado, que puede representarse en lenguaje natural mediante palabras (lemmas) muy diferentes: "perro", "dog", "can"...
 * un **lemma** es una palabra de lenguaje natural y, por lo tanto, puede tener varios significados (synsets).
 
Esta diferencia es importantísima tenerla presente.

Como vemos, a través de un synset llegamos a lemmas diferentes, pero todos ellos con el mismo significado. De hecho, el identificador del sysnset `dog.n.01` se mantiene:

In [8]:
ex_synset = wordnet.synset('dog.n.01')
for lemma in ex_synset.lemmas():
    print("{} -> {}".format(ex_synset, lemma))

Synset('dog.n.01') -> Lemma('dog.n.01.dog')
Synset('dog.n.01') -> Lemma('dog.n.01.domestic_dog')
Synset('dog.n.01') -> Lemma('dog.n.01.Canis_familiaris')


En cambio, cuando buscamos una palabra obtenemos varios lemmas, cada uno de ellos asociado a un synset diferente. Se puede observar cómo los identificadores de los synsets son diferentes: `dog.n.01`, `frump.n.01`...

In [9]:
ex_lemmas = wordnet.lemmas("dog")
for lemma in ex_lemmas:
    print("{} -> {} -> {}".format(lemma.name(), lemma, lemma.synset()))

dog -> Lemma('dog.n.01.dog') -> Synset('dog.n.01')
dog -> Lemma('frump.n.01.dog') -> Synset('frump.n.01')
dog -> Lemma('dog.n.03.dog') -> Synset('dog.n.03')
dog -> Lemma('cad.n.01.dog') -> Synset('cad.n.01')
dog -> Lemma('frank.n.02.dog') -> Synset('frank.n.02')
dog -> Lemma('pawl.n.01.dog') -> Synset('pawl.n.01')
dog -> Lemma('andiron.n.01.dog') -> Synset('andiron.n.01')
dog -> Lemma('chase.v.01.dog') -> Synset('chase.v.01')


## Relaciones

Como decíamos al principio, WordNet es más que un diccionario o un traductor, se trata de una **red de conceptos** que nos permite buscar relaciones entre significados de una forma extremadamente fácil e interesante.

### Synset y las relaciones semánticas

Los elementos de tipo Synset definen algunas relaciones que podemos explorar. Estas son las más interesantes (puedes consultar la lista completa [aquí](http://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset)):

 * hiperónimos
 * hipónimos
 * holónimos
 * merónimos
 
#### Hiperónimos e hipónimos
La hiperonimia e hiponimia codifican relaciones a nivel de significado. Un **hipónimo** concreta el significado de su **hiperónimo**; así, "mesa" es más específico que "mueble", y "altar" es un tipo particular de "mesa".

Podemos consultar los hiperónimos de un determinado synset con `.hypernyms()`:

In [10]:
data = []
for hypernym in my_synset.hypernyms():
    lemmas = [lemma.name() for lemma in hypernym.lemmas()]
    data.append([hypernym, ', '.join(lemmas), '</br>'.join(hypernym.examples())])
    
display_table(data, ["hyp-synset", "lemmas", "examples"], caption="Hyperónimos del {}".format(my_synset))

hyp-synset,lemmas,examples
Synset('copulate.v.01'),"copulate, mate, pair, couple",Birds mate in the Spring


Además de los hiperónimos a nivel de significado, también existen **a nivel de instancia (`instance_hypernyms`)** (igual con los hipónimos). Por ejemplo, si hemos encontrado en un texto una entidad ([NER](https://es.wikipedia.org/wiki/Reconocimiento_de_entidades_nombradas)) como `Vargas Llosa`, gracias a WordNet podemos hacer lo siguiente:

In [11]:
q = "Beethoven"  # Puedes probar con otros como p. ej.: Vargas_Llosa, Zweig, Einstein

ner = wordnet.synsets(q)[0]  # Recoge el primer resultado
hiperonimos = ner.instance_hypernyms()[0]  # Busca sus hiperónimos y quédate con el primero

print("Ha buscado: {}".format(q))
print("{} is a {}".format(ner.name(), hiperonimos.name()))  # Su hiperónimo me dice su profesión!!
print("Otras formas de llamarlo son: {}".format(', '.join([it.name() for it in ner.lemmas()])))
print("Su 'definición' es: {}".format(ner.definition()))

Ha buscado: Beethoven
beethoven.n.01 is a composer.n.01
Otras formas de llamarlo son: Beethoven, van_Beethoven, Ludwig_van_Beethoven
Su 'definición' es: German composer of instrumental music (especially symphonic and chamber music); continued to compose after he lost his hearing (1770-1827)


En estos casos, por ejemplo, el hiperónimo de un un escritor concreto es la categoría `escritor` puesto que generaliza su significado.

También podemos ver los hipónimos de un determinado synset:

In [12]:
data = []
for hyponym in my_synset.hyponyms():
    lemmas = [lemma.name() for lemma in hyponym.lemmas()]
    data.append([hyponym, ', '.join(lemmas), '</br>'.join(hyponym.examples())])
    
display_table(data, ["hyponym-synset", "lemmas", "examples"], caption="Hipónimos del {}".format(my_synset))

hyponym-synset,lemmas,examples
Synset('fornicate.v.01'),fornicate,
Synset('take.v.35'),"take, have",He had taken this woman when she was most vulnerable


#### Holónimos y merónimos
Las relaciones de holonimia y meronimia codifican relaciones entre el todo y sus partes. Un puerta (de coche) es una parte del coche.

In [13]:
my_synset = wordnet.synset("hand.n.01")

Puedes cambiar el valor almacenado en la variable `my_synset` para obtener otros resultados en las siguientes celdas. A lo mejor tienes que cambiar el método `.part_holonyms()` por otros métodos para averiguar otros tipos de holónimos, como `.member_holonyms()` o `.substance_holonyms()`.

In [14]:
data = []
for holonym in my_synset.part_holonyms():
    lemmas = [lemma.name() for lemma in holonym.lemmas()]
    data.append([holonym, ', '.join(lemmas), '</br>'.join(holonym.examples())])
    
display_table(data, ["holonym-synset", "lemmas", "examples"], caption="Holónimos del {}".format(my_synset))

holonym-synset,lemmas,examples
Synset('arm.n.01'),arm,
Synset('homo.n.02'),"homo, man, human_being, human",


A su vez, para ver los merónimos de un Synset, usaremos `.member_meronyms()`, `.substance_meronyms()`, `.part_meronyms()`, dependiendo del tipo de relación que tengan entre sí holónimo y merónimo.

In [15]:
data = []
for meronym in my_synset.part_meronyms():
    lemmas = [lemma.name() for lemma in meronym.lemmas()]
    data.append([meronym, ', '.join(lemmas), '</br>'.join(meronym.examples())])
    
display_table(data, ["meronym-synset", "lemmas", "examples"], caption="Merónimos del {}".format(my_synset))

meronym-synset,lemmas,examples
Synset('ball.n.10'),ball,the ball at the base of the thumbhe stood on the balls of his feet
Synset('digital_arteries.n.01'),"digital_arteries, arteria_digitalis",
Synset('finger.n.01'),finger,her fingers were long and thin
Synset('intercapitular_vein.n.01'),"intercapitular_vein, vena_intercapitalis",
Synset('metacarpal_artery.n.01'),"metacarpal_artery, arteria_metacarpea",
Synset('metacarpal_vein.n.01'),"metacarpal_vein, vena_metacarpus",
Synset('metacarpus.n.01'),metacarpus,
Synset('palm.n.01'),"palm, thenar",


### Los lemmas y sus relaciones
También podemos consultar las relaciones de los lemas. Las dos más interesantes (consulta la [lista completa](http://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Lemma)) son:

 - antónimos
 - formas derivadas

In [16]:
my_lemma = wordnet.lemma("fast.a.01.fast")

#### Antónimos

In [17]:
lang = 'eng'
data = []
for item in my_lemma.antonyms():
    lemmas = item.synset().lemma_names(lang=lang)
    data.append([item, '</br>'.join(lemmas), '</br>'.join(item.synset().examples())])
    
display_table(data, ["antonym-lemma", "lemmas", "examples"], caption="Antónimos del {}".format(my_lemma))

antonym-lemma,lemmas,examples
Lemma('slow.a.01.slow'),slow,a slow walkerthe slow lane of trafficher steps were slowhe was slow in reacting to the newsslow but steady growth


#### Formas derivadas

In [18]:
lang = 'spa'
data = []

for item in my_lemma.derivationally_related_forms():
    lemmas = item.synset().lemma_names(lang=lang)
    data.append([item, '</br>'.join(lemmas), '</br>'.join(item.synset().examples())])

display_table(data, ["derivationally-related-forms", "lemmas", "examples"],
              caption="Formas derivadas del {}".format(my_lemma))

derivationally-related-forms,lemmas,examples
Lemma('speed.n.02.fastness'),rapidezvelocidad,the project advanced with gratifying speed


## Aplicaciones

Las relaciones entre un elemento y sus vecinos permiten explorar la red de conceptos buscando términos relacionados con una palabra (lema) dada o bien con un concepto (synset) determinado. Sin embargo, esta estructura de relaciones nos permite también abordar problemas mucho más ambiciosos, como la expansión de búsquedas, la medición de la distancia semántica o la desambiguación.

### Expansión de búsquedas

La expansión de búsquedas es una de las técnicas que ha hecho que los buscadores hayan avanzado tanto en los últimos años. Consiste en generalizar a partir de la búsqueda del usuario a palabras que probablemente también le interesen porque están muy relacionadas semánticamente.

Para ello podemos utilizar estas redes de conceptos. Por ejemplo, si el usuario ha introducido `dalmatian` como término de búsqueda, podemos buscar synsets en los que aparece, expandir la búsqueda a todos los synsets relacionados y finalmente expandirla a todos sus hiperónimos:

In [19]:
q = "dalmatian"
print("Búsqueda original: {}".format(q))

# Synsets
synsets = wordnet.synsets(q)

# Lemas de otros synsets
def gather_lemmas(synset_list):
    lemmas = []
    for synset in synset_list:
        lemmas += synset.lemma_names()
    return lemmas
    
expanded_query = [q] + gather_lemmas(synsets)
print("Expandida a synsets: {}".format(', '.join(set(expanded_query))))

# Lemas de sus hiperónimos:
for synset in synsets:
    expanded_query += gather_lemmas(synset.hypernyms())

print("Expandida a hiperónimos: {}".format(', '.join(set(expanded_query))))

Búsqueda original: dalmatian
Expandida a synsets: carriage_dog, dalmatian, Dalmatian, coach_dog
Expandida a hiperónimos: carriage_dog, Dalmatian, domestic_dog, European, dog, coach_dog, Canis_familiaris, dalmatian


Al tener muchos más términos de búsqueda podremos recuperar muchos más documentos de nuestro corpus en caso de que la búsqueda original ofreciera un número insuficiente de resultados.

### Distancia semántica

Otra aplicación muy habitual cuando se dispone de una red de conceptos es medir la distancia semántica entre significados. Algunas situaciones en las que puede plantearse esta necesidad son la evaluación de traductores automáticos (cuál se ha separado menos del significado original) o la desambiguación (ante un mismo lema con varios significados, podemos asignarle una probabilidad a cada uno de ellos según la distancia a la temática del documento).

Esta aplicación es tan demandada que NLTK implementa los principales algoritmos para calcular esta medida. Por ejemplo, dados tres significados `dog.n.01`, `cat.n.01` y `tiger.n.01`, veamos cuál es la distancia entre ellos:

In [20]:
synset1 = wordnet.synset("dog.n.01")
synset2 = wordnet.synset("cat.n.01")
sim = wordnet.path_similarity(synset1, synset2)

print("La similitud entre el {} y el {} con este algoritmo es de {:0.4f}.".format(synset1, synset2, sim))

La similitud entre el Synset('dog.n.01') y el Synset('cat.n.01') con este algoritmo es de 0.2000.


(Con `{:0.4f}` lo que estamos haciendo es formatear el número de manera que nos muestre 4 decimales.)

Vamos a definir ahora una lista en las que incorporaremos todas las funciones que ofrece WordNet para el cálculo de la similaridad entre dos synsets (en las [referencias](#Referencias) se incluye un artículo en el que se detallan estos algoritmos):

In [21]:
# Métodos básicos para calcular similitud entre dos términos
methods = [('path_similarity', wordnet.path_similarity),
           ('Leacock-Chodorow', wordnet.lch_similarity),
           ('Wu-Palmer', wordnet.wup_similarity),]

# Algunos algoritmos necesitan un corpus para utilizarlo como referencia
from nltk.corpus import wordnet_ic, genesis
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
genesis_ic = wordnet.ic(genesis, False, 0.0) # Esto puede tardar un poco la primera vez

methods_ic = [('Resnik + Brown', lambda u,v: wordnet.res_similarity(u, v, brown_ic)),
              ('Resnik + Semcor', lambda u,v: wordnet.res_similarity(u, v, semcor_ic)),
              ('Resnik + Genesis', lambda u,v: wordnet.res_similarity(u, v, genesis_ic)),
                
              ('Jiang-Conrath + Brown', lambda u,v: wordnet.jcn_similarity(u, v, brown_ic)),
              ('Jiang-Conrath + Semcor', lambda u,v: wordnet.jcn_similarity(u, v, semcor_ic)),
              ('Jiang-Conrath + Genesis', lambda u,v: wordnet.jcn_similarity(u, v, genesis_ic)),
           
              ('Lin + Brown', lambda u,v: wordnet.lin_similarity(u, v, brown_ic)),
              ('Lin + Semcor', lambda u,v: wordnet.lin_similarity(u, v, semcor_ic)),
              ('Lin + Genesis', lambda u,v: wordnet.lin_similarity(u, v, genesis_ic)),]

Y también una función para mostrar los resultados de nuestros cálculos en una tabla:

In [22]:
float_format = "{0:.4f}"

def compute_distances(item_list, method_list):
    data = []
    it1 = item_list[0]
    for key,method in method_list:
        max_similarity = method(it1, it1)
        row = [float_format.format(1.0),]

        for it2 in item_list[1:]:
            similitud = method(it1, it2)/max_similarity  # Normalized by 'similarity(it1, it1)'
            row.append(float_format.format(similitud)) 
        data.append([key] + row)

    columns = ["{} - {}".format(item_list[0].name(), it.name()) for it in item_list]
    display_table(data, ["Método"] + columns, caption="Similitud normalizada entre '{}'".format(it1))

Con los elementos anteriores, la lista de algoritmos y la función auxiliar, podemos empezar a calcular cosas:

In [23]:
dog = wordnet.synset("dog.n.01")
cat = wordnet.synset("cat.n.01")
tiger = wordnet.synset("tiger.n.01")
animals = [dog, cat, tiger] # Aquí puedes añadir más synsets

compute_distances(animals, methods + methods_ic)

Método,dog.n.01 - dog.n.01,dog.n.01 - cat.n.01,dog.n.01 - tiger.n.01
path_similarity,1.0000,0.2000,0.1667
Leacock-Chodorow,1.0000,0.5576,0.5074
Wu-Palmer,1.0000,0.9231,0.7602
Resnik + Brown,1.0000,0.8785,0.2470
Resnik + Semcor,1.0000,0.9373,0.2355
Resnik + Genesis,1.0000,0.7236,0.1445
Jiang-Conrath + Brown,1.0000,0.0000,0.0000
Jiang-Conrath + Semcor,1.0000,0.0000,0.0000
Jiang-Conrath + Genesis,1.0000,0.0000,0.0000
Lin + Brown,1.0000,0.8768,0.2091


Como es de esperar, todos los algoritmos dan una similitud del 100 % para un synset comparado con sí mismo :D En cuanto a si consideramos un perro más parecido a un gato o a un tigre... parece que gana el gato, pero no siempre con holgura (mira los resultados que salen para Leacock-Chodorow).

Por supuesto, también es interesante hacerlo con entidades:

In [24]:
p1 = wordnet.synset("Einstein.n.01")
p2 = wordnet.synset("Austen.n.01")
p3 = wordnet.synset("Zweig.n.01")
p4 = wordnet.synset("Cousteau.n.01")
p5 = wordnet.synset("akhenaton.n.01")
people = [p1, p2, p3, p4, p5] # TODO: Aquí puedes añadir más synsets

for it in people:
    hyp = it.instance_hypernyms()[0]
    if hyp.lemmas()[0].name().startswith(("a", "e", "i", "o", "u")):
        print("{} is an {}".format(it.lemmas()[0].name(), hyp.lemmas()[0].name()))
    else:
        print("{} is a {}".format(it.lemmas()[0].name(), hyp.lemmas()[0].name()))

compute_distances([p1, p2, p3, p4, p5], methods)
compute_distances([p3, p1, p2, p4, p5], methods)

Einstein is a physicist
Austen is a writer
Zweig is a writer
Cousteau is an explorer
Akhenaton is a king


Método,einstein.n.01 - einstein.n.01,einstein.n.01 - austen.n.01,einstein.n.01 - zweig.n.01,einstein.n.01 - cousteau.n.01,einstein.n.01 - akhenaton.n.01
path_similarity,1.0000,0.1429,0.1429,0.1667,0.1250
Leacock-Chodorow,1.0000,0.4651,0.4651,0.5074,0.4283
Wu-Palmer,1.0000,0.6000,0.6000,0.6316,0.5714


Método,zweig.n.01 - zweig.n.01,zweig.n.01 - einstein.n.01,zweig.n.01 - austen.n.01,zweig.n.01 - cousteau.n.01,zweig.n.01 - akhenaton.n.01
path_similarity,1.0000,0.1429,0.3333,0.1667,0.1250
Leacock-Chodorow,1.0000,0.4651,0.6980,0.5074,0.4283
Wu-Palmer,1.0000,0.6000,0.6000,0.6316,0.5714


En general, Einstein se parece más a Cousteau que a Austen o a Zweig, y está más alejado de Akenatón (pero tampoco por mucho...). Sin embargo, Zweig claramente se parece más a Austen, porque ambos fueron escritores, que a los demás.

### Desambiguación

El problema clásico y ¿sin solución definitiva? del PLN es desambiguar el significado de una palabra. Una forma de abordarlo que podemos intentar con lo que sabemos hasta ahora es la siguiente:

In [25]:
sentence = "El banco presta dinero a cambio de un interés"

Sabemos tokenizar y realizar el análisis sintáctico de una oración. Con ello obtenemos las palabras individuales y la PoS de cada una de ellas:

In [26]:
sentence = [("bank", "n"), ("lend", "v"), ("money", "n"), ("interest", "n")]

# Eliminamos todo menos los nombres
sentence = [(it, pos) for it, pos in sentence if pos=="n"]
print(sentence)

[('bank', 'n'), ('money', 'n'), ('interest', 'n')]


Obviamente tenemos un problema de desambiguación puesto que cada una de estas palabras puede tener diferentes significados. Vamos a listarlos todos:

In [27]:
for word, pos in sentence:
    display(HTML("<strong>Significados de {}:</strong>".format(word)))

    synsets = wordnet.synsets(word, pos=pos)
    data = []
    for s in synsets:
        data.append([s, ', '.join([it.name() for it in s.lemmas()]), s.definition()])
        
    display_table(data)

Synset('bank.n.01'),bank,sloping land (especially the slope beside a body of water)
Synset('depository_financial_institution.n.01'),"depository_financial_institution, bank, banking_concern, banking_company",a financial institution that accepts deposits and channels the money into lending activities
Synset('bank.n.03'),bank,a long ridge or pile
Synset('bank.n.04'),bank,an arrangement of similar objects in a row or in tiers
Synset('bank.n.05'),bank,a supply or stock held in reserve for future use (especially in emergencies)
Synset('bank.n.06'),bank,the funds held by a gambling house or the dealer in some gambling games
Synset('bank.n.07'),"bank, cant, camber",a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
Synset('savings_bank.n.02'),"savings_bank, coin_bank, money_box, bank",a container (usually with a slot in the top) for keeping money at home
Synset('bank.n.09'),"bank, bank_building",a building in which the business of banking transacted
Synset('bank.n.10'),bank,a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)


Synset('money.n.01'),money,the most common medium of exchange; functions as legal tender
Synset('money.n.02'),money,wealth reckoned in terms of money
Synset('money.n.03'),money,the official currency issued by a government or national bank


Synset('interest.n.01'),"interest, involvement",a sense of concern with and curiosity about someone or something
Synset('sake.n.01'),"sake, interest",a reason for wanting something done
Synset('interest.n.03'),"interest, interestingness",the power of attracting or holding one's attention (because it is unusual or exciting etc.)
Synset('interest.n.04'),interest,a fixed charge for borrowing money; usually a percentage of the amount borrowed
Synset('interest.n.05'),"interest, stake",(law) a right or legal share of something; a financial involvement with something
Synset('interest.n.06'),"interest, interest_group",(usually plural) a social group whose members control some field of activity and who have common aims
Synset('pastime.n.01'),"pastime, interest, pursuit",a diversion that occupies one's time and thoughts (usually pleasantly)


La estrategia que vamos a seguir es probar todas las combinaciones posibles y quedarnos con aquella que ofrezca la mayor similitud entre sus componentes:

In [28]:
sentence_options = [wordnet.synsets(word, pos=pos) for word, pos in sentence]

import itertools
all_combinations = list(itertools.product(*sentence_options))
display(HTML("¡Hay {} combinaciones posibles!".format(len(all_combinations))))

data = {}
for comb in all_combinations:
    sim = 0
    for pair in itertools.combinations(comb, r=2):
        if pair[0].pos() == pair[1].pos():
            sim += wordnet.path_similarity(pair[0], pair[1])
    data[comb] = sim

import operator
sorted_data = sorted(data.items(), key=operator.itemgetter(1), reverse=True)

# Conservamos solo las primeras 10
display(HTML("Las 10 primeras son:"))
top10 = sorted_data[:10]

# Imprimimos los lemmas
data_to_display = []
for it, sim in top10:
    row = ["{:0.4f}".format(sim)]
    for synset in it:
        cell_text = "{}: {}".format(synset.name(), ', '.join([lema.name() for lema in synset.lemmas()]))
        row.append(cell_text)
    data_to_display.append(row)

display_table(data_to_display, headers=["score"] + [word for word, pos in sentence])

# Imprimimos el ganador
winner, sim = top10[0]
display(HTML("<strong>Y la ganadora, con una puntuación de {}, es:</strong><ul>".format(sim)))
for it in winner:
    display(HTML("<li><strong>{}</strong>: {}</li>".format(it.name(), it.definition())))
display(HTML("</ul>"))

score,bank,money,interest
0.4857,bank.n.06: bank,money.n.02: money,"interest.n.05: interest, stake"
0.3818,bank.n.06: bank,money.n.01: money,"interest.n.05: interest, stake"
0.3778,bank.n.04: bank,money.n.01: money,"interest.n.06: interest, interest_group"
0.3778,bank.n.04: bank,money.n.02: money,"interest.n.06: interest, interest_group"
0.3667,bank.n.06: bank,money.n.03: money,"interest.n.05: interest, stake"
0.3651,bank.n.06: bank,money.n.02: money,"interest.n.06: interest, interest_group"
0.3611,"depository_financial_institution.n.01: depository_financial_institution, bank, banking_concern, banking_company",money.n.01: money,"interest.n.06: interest, interest_group"
0.3611,"depository_financial_institution.n.01: depository_financial_institution, bank, banking_concern, banking_company",money.n.02: money,"interest.n.06: interest, interest_group"
0.3576,bank.n.04: bank,money.n.03: money,"interest.n.06: interest, interest_group"
0.3436,"depository_financial_institution.n.01: depository_financial_institution, bank, banking_concern, banking_company",money.n.03: money,"interest.n.06: interest, interest_group"


¡No está mal! Aunque es mejorable. ¿Cómo podemos mejorar este resultado de manera fácil? Si tenemos información sobre el contexto, por ejemplo, si sabemos que la oración aparece en una noticia de la sección económica de un periódico, entonces podemos utilizar como input la frecuencia relativa de nuestros significados dentro de ese corpus.

Pero eso ya para otro día...

## Referencias



 * [Global Wordnet Association](http://globalwordnet.org/): muy interesante la colección de links a wordnets en otros idiomas
 * [Universal Networking Language](https://en.wikipedia.org/wiki/Universal_Networking_Language): otra iniciativa de codificación unívoca del lenguaje. Uno de los grupos de desarrollo está en la UPM-Informática.
 * [«Medida de distancia semántica en grafos UNL»](https://www.dropbox.com/s/pygvolndftoshz9/GarciaSogo_Javier%20-%20Medida%20de%20distancia%20sem%C3%A1ntica%20en%20grafos%20UNL.pdf?dl=0) (Javier G. Sogo, 2015): utilizando UNL y WordNet se propone una metodología para evaluar la distancia semántica entre dos oraciones y así valorar el funcionamiento de dos traductores automáticos. Incluye documentación sobre los algoritmos de distancia semántica utilizados en la ontología de WordNet.